In [8]:
from decord import VideoReader, cpu
import numpy as np

In [5]:
def loadvideo_decord(sample, sample_rate_scale=1):
    """Load video content using Decord"""
    # pylint: disable=line-too-long, bare-except, unnecessary-comprehension
    fname = self.data_path + sample

    if not (os.path.exists(fname)):
        return []

    # avoid hanging issue
    if os.path.getsize(fname) < 1 * 1024:
        print('SKIP: ', fname, " - ", os.path.getsize(fname))
        return []
    try:
        if self.keep_aspect_ratio:
            vr = VideoReader(fname, num_threads=1, ctx=cpu(0))
        else:
            vr = VideoReader(fname, width=self.new_width, height=self.new_height,
                              num_threads=1, ctx=cpu(0))
    except:
        print("video cannot be loaded by decord: ", fname)
        return []

    converted_len = int(self.clip_len * self.frame_sample_rate)
    seg_len = len(vr) // self.num_segment

    all_index = []
    for i in range(self.num_segment):
        if seg_len <= converted_len:
            index = np.linspace(0, seg_len, num=seg_len // self.frame_sample_rate)
            index = np.concatenate((index, np.ones(self.clip_len - seg_len // self.frame_sample_rate) * seg_len))
            index = np.clip(index, 0, seg_len - 1).astype(np.int64)
        else:
            end_idx = np.random.randint(converted_len, seg_len)
            str_idx = end_idx - converted_len
            index = np.linspace(str_idx, end_idx, num=self.clip_len)
            index = np.clip(index, str_idx, end_idx - 1).astype(np.int64)
        index = index + i*seg_len
        all_index.extend(list(index))

    all_index = all_index[::int(sample_rate_scale)]
    vr.seek(0)
    buffer = vr.get_batch(all_index).asnumpy()
    return buffer

In [7]:
vid_path = '/home/aiteam/tykim/video_recog/dataset/mix_dataset/train/train_1086.mp4'
vr = VideoReader(vid_path, num_threads=1, ctx=cpu(0))
# buf = loadvideo_decord(vid_path)

In [11]:
buffer = vr.get_batch([1,2,3,4,5]).asnumpy()

In [13]:
buffer.shape

(5, 720, 1280, 3)

In [ ]:
/home/aiteam/tykim/video_recog/gluon-cv/gluoncv/torch/data/video_cls/dataset_classification.py



for ..gluon-cv import glu